In [ ]:
import json
import random
import numpy as np
import pickle

In [ ]:
import nltk

In [ ]:
from nltk.stem import WordNetLemmatizer

In [ ]:
from keras.models import load_model

In [ ]:
lemmatizer = WordNetLemmatizer()

In [ ]:
with open('C:/Users/Makro_1997/json file/intents.json') as f:
    intents = json.load(f)

In [ ]:
words = pickle.load(open('words.pkl','rb'))
classes = pickle.load(open('classes.pkl','rb'))
model = load_model('chatbot model.model')

In [ ]:
def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word) for word in sentence_words]
    return sentence_words

In [ ]:
def bag_of_words(sentence):
    sentence_words = clean_up_sentence(sentence)
    bag = [0]* len(words)
    for w in sentence_words:
        for i, word in enumerate(words):
            if word == w:
                bag[i] = 1
    return np.array(bag)

In [ ]:
def predict_class(sentence):
    bow = bag_of_words(sentence)
    res = model.predict(np.array([bow]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i, r in enumerate(res) if r>ERROR_THRESHOLD]
    
    results.sort(key=lambda x: x[1],reverse=True)
    return_list = []
    for r in results:
        return_list.append({'intent':classes[r[0]],'probablity':str(r[1])})
    return return_list

In [ ]:
def get_response(intents_list,intents_json):
    tag = intents_list[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if i['tag'] == tag:
            result = random.choice(i['responses'])
            break
    return result
        

In [ ]:
print('GO! Bot is running')
while True:
    message = input("")
    ints = predict_class(message)
    res = get_response(ints,intents)
    print(res)
